In [14]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
import faiss
import numpy as np

In [16]:
docs = [
    """Mustafa Kemal Atatürk was born in Thessaloniki in 1881 and became the founder of the Republic of Turkey.
    In 1923, he declared the Republic and laid the foundations of modern Turkey.""",

    """Ankara is the capital of Turkey and is located in the Central Anatolia Region. It was declared the capital on October 13, 1923.""",

    """The Python programming language is an open-source, high-level and object-oriented language.
    It is widely used in areas such as machine learning and data science."""


]


In [17]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(docs, convert_to_numpy=True)


In [18]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
question_embedding = model.encode([question], convert_to_numpy=True)
D, I = index.search(question_embedding, k=1)
relevant_doc = docs[I[0][0]]


In [19]:
question = "Who is the founder of the Republic of Turkey??"
question_embedding = model.encode([question], convert_to_numpy=True)

D, I = index.search(question_embedding, k=1)
relevant_doc = docs[I[0][0]]
print("En alakalı bilgi:", relevant_doc)


En alakalı bilgi: Mustafa Kemal Atatürk was born in Thessaloniki in 1881 and became the founder of the Republic of Turkey.
    In 1923, he declared the Republic and laid the foundations of modern Turkey.


In [22]:


model_name = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)
tokenizer.pad_token = tokenizer.pad_token or tokenizer.eos_token


C:\Users\mehmetemre\.conda\envs\rag-gpu\lib\site-packages\accelerate\utils\modeling.py:1569: UserWarning: Current model requires 2112 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.65it/s]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [23]:
# Soru + bilgiyle prompt hazırlanıyor
prompt = f"""
Soru: {question}
Bilgi: {relevant_doc}
Cevap:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # GPU'ya gönder
outputs = model.generate(**inputs, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Soru: Who is the founder of the Republic of Turkey??
Bilgi: Mustafa Kemal Atatürk was born in Thessaloniki in 1881 and became the founder of the Republic of Turkey.
    In 1923, he declared the Republic and laid the foundations of modern Turkey.
Cevap:
Mustafa Kemal Atatürk was born in Thessaloniki in 1881 and became the founder of the Republic of Turkey.
